# Continuing from last time, we will try to improve performance on our CXR dataset using a DenseNet model that have been pretrained on CXR
## Preparation instructions
1. Make sure you have a Google account
2. Copy **this notebook** and **L12_pretrained_densenet121_NIH.h5** files to the root directory of your Google Drive

## Remember to change RUNTIME TYPE to GPU

## To mount your Google Drive to Google Colab, run the command below and follow the instruction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Check GPU status

In [ ]:
!nvidia-smi

## Install UMAP

In [ ]:
!pip install umap-learn

## Import libraries
Google Colab should already have these basic libraries installed

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

import tensorflow as tf
import tensorflow.keras.layers as layers

import umap
import seaborn as sns

## First, let's recap on the two ways to create model in TensorFlow/Keras
**tensorflow.keras.models.Model** and **tensorflow.keras.models.Sequential**

### With Sequential interface, we define a model and then add layers to it one a time in sequential order
Note that the input is **layers.InputLayer(input_shape = (128, 128, 1))**

In [ ]:
def generate_sequential_model():
    model = Sequential()
    model.add(layers.InputLayer(input_shape = (128, 128, 1)))

    model.add(layers.Conv2D(filters = 64, kernel_size = [5, 5], padding = 'valid', strides = [2, 2]))
    model.add(layers.Activation(activation = 'relu'))

    model.add(layers.Conv2D(filters = 128, kernel_size = [3, 3], padding = 'valid', strides = [1, 1]))
    model.add(layers.Activation(activation = 'relu'))
    model.add(layers.MaxPool2D(pool_size = [3, 3], strides = [2, 2]))

    model.add(layers.Flatten())

    model.add(layers.Dense(units = 256, activation = 'relu'))
    model.add(layers.Dense(units = 256, activation = 'relu'))
    model.add(layers.Dense(units = 7, activation = 'softmax'))
    
    return model

In [ ]:
ex_sequential_model = generate_sequential_model()
ex_sequential_model.summary()

### With Model interface, we connect layers manually and then call Model() to designate the input and output layers
Note that the input is **layers.Input(shape = (128, 128, 1))**

In [ ]:
def generate_model():
    inputs = layers.Input(shape = (128, 128, 1))
    x = layers.Conv2D(filters = 64, kernel_size = [5, 5], padding = 'valid', strides = [2, 2])(inputs)
    x = layers.Activation(activation = 'relu')(x)
    
    x = layers.Conv2D(filters = 128, kernel_size = [3, 3], padding = 'valid', strides = [1, 1])(x)
    x = layers.Activation(activation = 'relu')(x)
    x = layers.MaxPool2D(pool_size = [3, 3], strides = [2, 2])(x)

    x = layers.Flatten()(x)

    x = layers.Dense(units = 256, activation = 'relu')(x)
    x = layers.Dense(units = 256, activation = 'relu')(x)
    outputs = layers.Dense(units = 7, activation = 'softmax')(x)

    return Model(inputs = inputs, outputs = outputs)

In [ ]:
ex_model = generate_model()
ex_model.summary()

### The power of Model over Sequential is that we can define model with multiple branches

In [ ]:
def generate_multi_branch_model():
    inputs1 = layers.Input(shape = (128, 128, 1), name = 'image_data_input')
    x1 = layers.Conv2D(filters = 64, kernel_size = [5, 5], padding = 'valid', strides = [2, 2])(inputs1)
    x1 = layers.Activation(activation = 'relu')(x1) 
    x1 = layers.Conv2D(filters = 128, kernel_size = [3, 3], padding = 'valid', strides = [1, 1])(x1)
    x1 = layers.Activation(activation = 'relu')(x1)
    x1 = layers.MaxPool2D(pool_size = [3, 3], strides = [2, 2])(x1)
    x1 = layers.Flatten()(x1)

    inputs2 = layers.Input(shape = (100), name = 'clinical_data_input')
    x2 = layers.Dense(units = 128, activation = 'relu')(inputs2)

    merged = layers.Concatenate()([x1, x2])

    merged1 = layers.Dense(units = 256, activation = 'relu')(merged)
    merged1 = layers.Dense(units = 256, activation = 'relu')(merged1)
    outputs1 = layers.Dense(units = 7, activation = 'softmax', name = 'classification_output')(merged1)

    merged2 = layers.Dense(units = 128, activation = 'relu')(merged)
    outputs2 = layers.Dense(units = 1, activation = 'linear', name = 'regression_output')(merged2)

    return Model(inputs = [inputs1, inputs2], outputs = [outputs1, outputs2])

In [ ]:
ex_multi_branch_model = generate_multi_branch_model()
ex_multi_branch_model.summary()

## We can use plot_model function from tensorflow.keras.utils to get a clearer picture of this architecture

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(ex_multi_branch_model, to_file = 'model.png', show_shapes = True, show_layer_names = True)

## Now, let's get back to our chest x-ray data
This file is in a compressed **h5** format. We can read it with library **h5py** which is already available on Google Colab server

### We will extract only images with single diagnosis label

In [ ]:
cxr_data = []
cxr_label = []

with h5py.File('/content/drive/MyDrive/L11.2_CXR_data.h5', 'r') as data:
    findings = [x.decode('UTF-8') for x in data['Finding Labels']]
    images = data['images']
    
    for i in range(len(findings)):
        if len(findings[i].split('|')) == 1: ## single diagnosis
            if findings[i] == '':
                cxr_label.append('Normal')
            else:
                cxr_label.append(findings[i])
            
            cxr_data.append(images[i] / 255.0) ## divide by 255.0
            
cxr_data = np.array(cxr_data)
cxr_label = np.array(cxr_label)

### The image data has already been formatted for CNN input (notice the last channel dimension)

In [ ]:
print('data dimension:', cxr_data.shape)
print('label dimension:', cxr_label.shape)

### We just need to format the label into one-hot code
Use **sklearn.preprocessing** module's **OneHotEncoder()**

In [ ]:
encoder = OneHotEncoder(sparse = False).fit(cxr_label.reshape(-1, 1)) ## reshape(-1, 1) to convert 1D vector into 2D
cxr_label_onehot = encoder.transform(cxr_label.reshape(-1, 1))

print('onehot label dimension:', cxr_label_onehot.shape)
print('categories:', encoder.categories_[0])

### There are 15 classes: 14 diagnosis + normal
Let's look at the class distribution

In [ ]:
h, bins = np.histogram(np.argmax(cxr_label_onehot, axis = 1), bins = cxr_label_onehot.shape[1])
plt.bar(range(cxr_label_onehot.shape[1]), h)
plt.xticks(range(cxr_label_onehot.shape[1]), labels = encoder.categories_[0], rotation = 90)
plt.show()

### Some classes, like Hernia and Pneumonia, have very few images
## Let's select classes with plenty of data

In [ ]:
selected_classes = ['Atelectasis', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Normal', 'Pneumothorax']
selected_classes_cols = [i for i in range(len(encoder.categories_[0])) if encoder.categories_[0][i] in selected_classes]

selected_label_onehot = cxr_label_onehot[:, selected_classes_cols]
selected_row = selected_label_onehot.sum(axis = 1) == 1

selected_data = cxr_data[selected_row, :]
selected_label_onehot = selected_label_onehot[selected_row, :]

print('selected data dimension:', selected_data.shape)
print('selected label dimension:', selected_label_onehot.shape)

## Duplicate the data to create 3-channel inputs

In [ ]:
selected_data_3ch = np.concatenate([selected_data, selected_data, selected_data], axis = 3)
print(selected_data_3ch.shape)

## Split selected data into 60-20-20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selected_data_3ch, selected_label_onehot, test_size = 0.2, stratify = selected_label_onehot, \
                                                    shuffle = True, random_state = 3011979)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify = y_train, \
                                                  shuffle = True, random_state = 3011979)

## Delete unused variables to save RAM space

In [ ]:
del selected_data
del selected_data_3ch

## Create a DenseNet121 model that fits the pretrained model's description
The model was trained on 14-class CXR dataset using the following architecture
1. DenseNet121
2. GlobalAveragePooling2D
3. Dense with 1024 units and ReLU activation
4. BatchNormalization
5. Dropout with rate 0.2
6. Dense with 512 units and ReLU activation
7. BatchNormalization
8. Dropout with rate 0.2
9. Dense with 14 units and sigmoid activation

In [ ]:
def generate_cxr_pretrained_densenet():
    base_model = DenseNet121(input_shape = (128, 128, 3), weights = None, classes = 14, include_top = False)
    base_model.trainable = False ## freeze convolutional part

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(1024, activation = 'relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(512, activation = 'relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(14, activation = 'sigmoid')(x)
    model = Model(inputs = base_model.input, outputs = x)

    return model

In [ ]:
cxr_pretrained_densenet = generate_cxr_pretrained_densenet()
cxr_pretrained_densenet.summary()

## After confirming the architecture, let's load the pretrained weights from **densenet_121_ADAM_14_class_lr=0.01_avgpool_224_NIH.h5**

In [ ]:
cxr_pretrained_densenet.load_weights('/content/drive/MyDrive/L12_pretrained_densenet121_NIH.h5')

## However, this model architecture doesn't exactly match what we want
Our task only contain 7 outputs, not 14

## We can access each **layer** in the model through **model.layers**

In [ ]:
print('Layer at position 0:', cxr_pretrained_densenet.layers[0].name, cxr_pretrained_densenet.layers[0])
print('Layer at position -2:', cxr_pretrained_densenet.layers[-2].name, cxr_pretrained_densenet.layers[-2])
print('Layer at position -1:', cxr_pretrained_densenet.layers[-1].name, cxr_pretrained_densenet.layers[-1])

## Let's create a new model that is identical to the pretrained one, except that the output layer changes from Dense(14) to Dense(7)
The trick is to connect the last Dropout layer (at position -2) to a new Dense(7) layer

Note that we are also changing the formulation of CXR classification from **multi-class classification** to **multi-label classification**. This changes the final layer's activation from **softmax** to **sigmoid**

In [ ]:
new_output = layers.Dense(7, activation = 'sigmoid')(cxr_pretrained_densenet.layers[-2].output)
new_cxr_pretrained_densenet = Model(inputs = cxr_pretrained_densenet.input, outputs = new_output)
new_cxr_pretrained_densenet.summary()

### Note that the parent model's weights and settings are inherited

## Let's do some inference using the loaded models

In [ ]:
y_val_pred = new_cxr_pretrained_densenet.predict(X_val)

for i in range(len(selected_classes)):
    print('class:', selected_classes[i], 'accuracy:', accuracy_score(y_val[:, i], y_val_pred[:, i] > 0.5))

## Define callbacks

In [ ]:
modelckpt_callback = ModelCheckpoint('/content/drive/MyDrive/densenet_cxr_L12_checkpoint/', \
                                     verbose = 1, save_best_only = True, save_weights_only = True)
tensorboard_callback = TensorBoard('/content/drive/MyDrive/tb_log_L12', histogram_freq = 1)
reduced_lr_callback = ReduceLROnPlateau(patience = 3, factor = 0.2)

## Start TensorBoard iinstance

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/tb_log_L12

## Define ImageDataGenerator to apply some augmentations

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    )

datagen.fit(X_train)

## Compile and train the model


In [ ]:
new_cxr_pretrained_densenet.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate = 1e-4))

In [ ]:
tf.random.set_seed(3011979)
history = new_cxr_pretrained_densenet.fit(datagen.flow(X_train, y_train, batch_size = 128), steps_per_epoch = X_train.shape[0] / 128,
                                          validation_data = (X_val, y_val), epochs = 20,
                                          callbacks = [modelckpt_callback, tensorboard_callback, reduced_lr_callback], verbose = 1, shuffle = True)

## There is no good built-in metric for multi-label classification, so we can only monitor the loss values

## Evaluate model performance on the validation set again

In [ ]:
y_val_pred = new_cxr_pretrained_densenet.predict(X_val)

for i in range(len(selected_classes)):
    print('class:', selected_classes[i], 'accuracy:', accuracy_score(y_val[:, i], y_val_pred[:, i] > 0.5))

## And on the test set

In [ ]:
y_test_pred = new_cxr_pretrained_densenet.predict(X_test)

for i in range(len(selected_classes)):
    print('class:', selected_classes[i], 'accuracy:', accuracy_score(y_test[:, i], y_test_pred[:, i] > 0.5))

## Next, let's explore the embedding of CXR images learned by our model
This is the output of the **global_average_pooling2d** layer in our model

### To obtain the output of this specific layer, we use the same trick as when we replace Dense(14) with dense(7)
Create a new model with the same input but with output being the **global_average_pooling2d**'s output.

We can directly call a specific layer using its name via **model.get_layer(name = '')**

In [ ]:
gap_output = new_cxr_pretrained_densenet.get_layer(name = 'global_average_pooling2d').output
embedding_model = Model(inputs = new_cxr_pretrained_densenet.input, outputs = gap_output)

X_test_embedding = embedding_model.predict(X_test)

### The embedding should have dimension = (number of test samples, 1024)

In [ ]:
X_test_embedding.shape

## Use UMAP to visualize the embedding distribution

In [ ]:
X_test_embedding_umap = umap.UMAP(n_neighbors = 50, min_dist = 0.5, n_components = 2, metric = 'euclidean', random_state = 3011979).fit_transform(X_test_embedding)

### Colored by predicted probability

In [ ]:
plt.figure(figsize = (17, 8))

for i in range(len(selected_classes)):
    plt.subplot(2, 4, i + 1)
    plt.scatter(X_test_embedding_umap[:, 0], X_test_embedding_umap[:, 1], c = y_test_pred[:, i], cmap = 'Reds', alpha = 0.8)
    plt.title(selected_classes[i]); plt.colorbar()

plt.tight_layout()
plt.show()

### Colored by ground truth label

In [ ]:
plt.figure(figsize = (16, 8))

for i in range(len(selected_classes)):
    plt.subplot(2, 4, i + 1)
    sns.kdeplot(x = X_test_embedding_umap[y_test[:, i] == 1, 0], y = X_test_embedding_umap[y_test[:, i] == 1, 1], fill = True)
    plt.title(selected_classes[i])

plt.tight_layout()
plt.show()